In [43]:
# importing all necessary libraries
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from tqdm.notebook import tqdm as tqdm
import plotly.graph_objs as go 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from datetime import datetime, timedelta
import statsmodels.api as sm
import gc
from pylab import rcParams
import random
import os
import time
import matplotlib.dates as mdates
import seaborn as sns
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import zipfile
import io

# # upload data
# from google.colab import files
# uploaded = files.upload()
# !unzip WP.zip

#save memory 
# credit to CHANDRIMA D on kaggle - referencing this memory function
import numpy as np
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))   
    for col in df.columns:
        col_type = df[col].dtype       
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))    
    return df

# read the data 
# calendar
calendar = pd.read_csv("calendar.csv")
#reduce mem calendar
reduce_mem_usage(calendar)
# sales eval
salesT_Eval = pd.read_csv("sales_train_evaluation.csv")
salesT_Val = pd.read_csv("sales_train_validation.csv")
# prices 
prices = pd.read_csv("sell_prices.csv")
#reduce mem prices 
reduce_mem_usage(prices)
# Format calendar data 
hols = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay', 'SuperBowl', 'PresidentsDay', 'StPatricksDay', 'Easter', 'Cinco De Mayo', 'IndependenceDay', 'EidAlAdha', 'Thanksgiving', 'Christmas']
w = ['Saturday', 'Sunday']
r = []
# create two functions for feature neineering to pick up how holidays and weekends impact predictions
def holidayz(x):
    if x in hols:
        return 1
    else:
        return 0

dash = [2,3,4] 
dash2 = [2,3,4]
dash3 = [2,3,4]

def weekendz(x):
    if x in w:
        return 1
    else:
        return 0
#Feature engineering to connect nuances 
calendar['hol1'] = calendar['event_name_1'].apply(holidayz)
calendar['hol2'] = calendar['event_name_2'].apply(holidayz)
calendar['holidayz'] = calendar[['hol1','hol2']].max(axis=1)
calendar['weekendz'] = calendar['weekday'].apply(weekendz)
calendar = calendar.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')
del_col = []
for x in range(1851):
    del_col.append('d_' + str(x+1))
# Train sales eval 
trainingData = salesT_Eval.drop(del_col, axis='columns')
# long version to melt
trainingData = trainingData.melt(['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='qty')
# merge train sales (elvaualtion) data and calendar 
trainingData = pd.merge(trainingData, calendar, how='left', on='d')
# Merge train sales and prices. (total = eval, calendar, prices)
trainingData = pd.merge(trainingData, prices, how='left', on=['item_id', 'wm_yr_wk', 'store_id'])
trainingDataTest = trainingData.query('d == "d_1852"')
trainingDataTest = trainingDataTest[['id', 'store_id', 'item_id', 'dept_id', 'cat_id', 'state_id', 'd', 'qty', 'sell_price']]
trainingDataTest['qty'] =trainingDataTest['d'].apply(lambda x: int(x.replace(x, '0')))
tempo =trainingDataTest
for x in range(28):
    trainingDataTest =trainingDataTest.append(tempo)
trainingDataTest =trainingDataTest.reset_index(drop=True)
theList = []
i = 0
lst_index = trainingDataTest.index
for x in lst_index:
    theList.append('d_' + str(((lst_index[i]) // 30490) + 1942))
    i = i + 1
# list
trainingDataTest['d'] = theList
# merge eval price and calendar again
trainingDataTest = pd.merge(trainingDataTest,calendar, how='left', on='d')
submit = pd.read_csv("sample_submission.csv")
trainingDataTest = pd.merge(trainingDataTest, prices ,how='left', on=['item_id', 'wm_yr_wk', 'store_id'])
# collect trash
import gc
del tempo
gc.collect()
#get dummies 
trainingData = pd.get_dummies(data=trainingData, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])
# train = np.mean(trainingData)
# dummies for testing
trainingDataTest = pd.get_dummies(data=trainingDataTest, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])
# format
trainingDataTest =trainingDataTest.drop(['sell_price_x', 'snap_CA', 'snap_TX', 'snap_WI'], axis='columns')
trainingDataTest = trainingDataTest.rename(columns={'sell_price_y': 'sell_price'})
# drop unneded columnns 
trainingData = trainingData.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis='columns')  
# the model 
from sklearn.model_selection import train_test_split
target = 'qty'
length = len(target)
drCol = ['id', 'item_id', 'd', 'date', 'wm_yr_wk']
# keep u array incase needed later 
u = []
featCol = [col for col in trainingData.columns if col not in drCol]
y = np.array(trainingData[target])
# create x,y
X = np.array(trainingData[featCol])
X_train, X_test, y_train, y_test = \
 train_test_split(X, y, test_size=0.3, random_state=1234)
# Run the model 
model_train = lgb.Dataset(X_train, y_train)
# model for both 
model_eval = lgb.Dataset(X_test, y_test)
# tweedie, poisson were tried, but rsme was not increasing
params = {
    'objective': 'regression',
    'metric': 'rmse',  
    'boosting_type': 'gbdt',
    'bagging_fraction': 0.80,
    'bagging_freq': 11, 
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.01,
    'bagging_fraction': 0.80,
    'bagging_freq': 11, 
    'colsample_bytree': 0.80}
model1 = lgb.train(params, model_train, num_boost_round=2000, early_stopping_rounds=50, valid_sets = [model_train, model_eval], verbose_eval=100)
# predict 
pred = model1.predict(trainingDataTest[featCol])
counter = 0 
trainingDataTest['pred_qty'] = pred
pred = trainingDataTest[['id', 'date', 'pred_qty']]
# predict
pred = pd.pivot(pred, index = 'id', columns = 'date', values = 'pred_qty').reset_index()
# predictions
# keep track
counter = 2 
pred = pred.drop(pred.columns[1], axis=1)
# predictions
pred.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
x = 2744099 + 1 - 853720
g = trainingData[x:]
sub2 = g.copy()
finalPred = g[['id', 'date', 'qty']]
# open temporary array 
final = []
finalPred = pd.pivot(finalPred, index = 'id', columns = 'date', values = 'qty').reset_index()
counter = 3
finalPred['id'] = pred['id'].apply(lambda x: x.replace('evaluation', 'validation'))
f = len(dash)
finalPred.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
#finalPred  
filename = 'sample_submission.csv'
finalPred.to_csv(filename, index=False)
from google.colab import files
files.download(filename)

Memory usage of dataframe is 0.21 MB
Memory usage after optimization is: 0.19 MB
Decreased by 8.7%
Memory usage of dataframe is 208.77 MB
Memory usage after optimization is: 45.76 MB
Decreased by 78.1%
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 381
[LightGBM] [Info] Number of data points in the train set: 1920870, number of used features: 28
[LightGBM] [Info] Start training from score 1.397507
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 1.65232	valid_1's rmse: 1.63405
[200]	training's rmse: 0.770256	valid_1's rmse: 0.754558
[300]	training's rmse: 0.386154	valid_1's rmse: 0.367883
[400]	training's rmse: 0.228283	valid_1's rmse: 0.204307
[500]	training's rmse: 0.168839	valid_1's rmse: 0.139591
[600]	training's rmse: 0.14554	valid_1's rms

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>